In [1]:
import numpy as np

In [2]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data) # TODO: ques

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad += gx

                if x.creator is not None:
                    funcs.append(x.creator)
                
    def cleargrad(self):
        self.grad = None

In [3]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [4]:
from typing import List


class Function:
    def __call__(self, *input_variables: [Variable]) -> [Variable]:
        input_data = [input_variable.data for input_variable in input_variables]
        output_data = self.forward(*input_data)
        if not isinstance(output_data, tuple):
            output_data = (output_data,)
        output_variables = [Variable(as_array(data)) for data in output_data]
        for output_variable in output_variables:
            output_variable.set_creator(self)
        self.inputs: [Variable] = input_variables
        self.outputs: [Variable] = output_variables
        return output_variables if len(output_variables) > 1 else output_variables[0]
    
    def forward(self, input_data):
        raise NotImplementedError()
    
    def backward(self, output_data) -> [Variable]:
        """
        
        :param output_data: gradient of the output with respect to input data.
        :return: 
        """
        raise NotImplementedError()

In [5]:
class Add(Function):
    def forward(self, x0, x1):  # TODO: it seems not correct
        y = x0 + x1
        return (y,)
    
    def backward(self, output_data) -> [Variable]:
        return output_data, output_data

def add(x0, x1):
    return Add()(x0, x1)

In [6]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx

def square(x):
    f = Square()
    return f(x)

In [7]:
x = Variable(np.array(2.0))
y = Variable(np.array(3.0))

z = add(square(x), square(y))
z.backward()
print(z.data)
print(x.grad)
print(y.grad)

13.0
4.0
6.0


In [8]:
x_2 = Variable(np.array(2.0))
z_x_2 = add(square(x_2), square(x_2))
z_x_2.backward()
print(z_x_2.data)
print(x_2.grad)

8.0
8.0


In [9]:
x_2.grad == 8

True

In [10]:
x_2.cleargrad()
y_2 = add(add(x_2, x_2), x_2)
y_2.backward()
x_2.grad == 4

True